# Save mds vector output with time series of all ADCP longitudes/depths

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from xmitgcm import open_mdsdataset, utils
from MITgcmutils import rdmds, wrmds
import xarray as xr
from datetime import datetime, timedelta
import pych.ecco as pe

In [2]:
llc = pe.read_mds('/workspace/grids/llc90',iters=None)

/workspace/xmitgcm/xmitgcm/mds_store.py:854: UserWarning: Couldn't find available_diagnostics.log in /workspace/grids/llc90 or /workspace/grids/llc90. Using default version.
  "in %s or %s. Using default version." % (data_dir, grid_dir))


In [15]:
def get_single_timeseries(lon,klev,lon_list):
    lonoffset = lon_list.index(lon)
    kgen = lonoffset*22 + klev + 1
    return np.squeeze(rdmds(f'../mdsfiles/uvel_all_lons/euc_out.{kgen:03}.0000000001'))

In [16]:
def get_single_longitude(lon,lon_list):
    all_depths = []
    for k in range(22):
        all_depths.append(get_single_timeseries(lon,k,lon_list))
    return np.array(all_depths).T

In [17]:
def get_all_longitudes(londict,lon_list):
    all_lons = []
    for key,val in londict.items():
        arr = get_single_longitude(val,lon_list)
        all_lons.append(arr)
        
    return np.stack(all_lons,axis=2)

In [18]:
def get_uvel(arr=None):
    
    lonstrs = ['110W','140W','170W','165E','156E','147E']
    longs = [-110,-140,-170,165,156,147]
    lond = dict(zip(lonstrs,longs))
    arr = get_all_longitudes(lond,longs) if arr is None else arr
    
    ntime = arr.shape[0]
    ecco_time = np.array([np.datetime64(x) for x in datetime(1992,1,1,12)+(np.arange(ntime)+1)*timedelta(hours=1)])
    time = xr.DataArray(ecco_time,coords={'time':ecco_time},dims=('time',),name='time')
    longitude = xr.DataArray(np.array(longs),coords={'lon':np.array(longs)},dims=('lon',),name='lon')
    z = llc['Z'][:22].copy().swap_dims({'k':'Z'})
    depth = -z.rename({'Z':'depth'})
    depth.attrs['long_name'] = 'Depth'
    depth.attrs['units'] = 'm'
    
    uvel = xr.DataArray(arr,coords={'time':time,'depth':depth,'lon':longitude},dims=('time','depth','lon'),
                        attrs={'descrption':'Hourly Zonal Velocity',
                               'units':'m/s'})
    
    return uvel

In [19]:
uvel = get_uvel()

In [24]:
fname = '../ncfiles/eccov4r4_uvel.nc'

In [25]:
uvel.to_netcdf(fname)